In [1]:
from z3 import *

def check(f1, f2, f3):
    def __check(f):
        s = Solver()
        s.add(Not(f))
        res = s.check()
        print(res)
        if res == sat:
            print(s.model())
    
    __check(f1)
    __check(f2)
    __check(f3)


# loop-zilu

In [3]:
# loop-zilu/benchmark03_linear.c
#   x = 0; y = 0;
#   if (!(i==0 && j==0)) return 0;
#   while (__VERIFIER_nondet_bool()) {
#     x++;
#     y++;
#     i+=x;
#     j+=y;
#     if (flag) j+=1;
#   }
#   __VERIFIER_assert(j>=i);

i = Int('i')
j = Int('j')
x = Int('x')
y = Int('y')
ip = Int('ip')
jp = Int('jp')
xp = Int('xp')
yp = Int('yp')
flag = Bool('flag')

P = And(x==0, y==0, i==0, j==0)
Q = j>=i
T = And(xp==x+1, yp==y+1, ip==i+xp, Implies(Not(flag), jp==j+yp), Implies(flag, jp==j+1+yp))
I = lambda i,j,x,y: And(-i+j>=0, x-y<=0)
    # And(j==k, j>=0, n > 0)

F1 = Implies(P, I(i,j,x,y))
F2 = Implies(And(I(i,j,x,y), T), I(ip,jp,xp,yp))
F3 = Implies(I(i,j,x,y), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [4]:
#   loop-zilu/benchmark04_conjunctive.c
#   if (!(n>=1 && k>=n && j==0)) return 0;
#   while (j<=n-1) {
#     j++;
#     k--;
#   }
#   __VERIFIER_assert(k>=0);

n = Int('n')
k = Int('k')
j = Int('j')
jp = Int('jp')
kp = Int('kp')

P = And(n>=1, k>=n, j==0)
Q = k>=0
T = And(jp==j+1, kp==k-1)
B = j<=n-1
I = lambda n,k,j: And(j+k>=n, j<=n)

F1 = Implies(P, I(n,k,j))
F2 = Implies(And(I(n,k,j), B, T), I(n,kp,jp))
F3 = Implies(I(n,k,j), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [11]:
# loop-zilu/benchmark05_conjunctive.c
#   if (!(x>=0 && x<=y && y<n)) return 0;
#   while (x<n) {
#     x++;
#     if (x>y) y++;
#   }
#   __VERIFIER_assert(y==n);

x = Int('x')
y = Int('y')
n = Int('n')
xp = Int('xp')
yp = Int('yp')

P = And(x>=0, x<=y, y<n)
Q = y==n
B = x<n
T = And(xp==x+1, Implies(xp>y, yp==y+1), Implies(Not(xp>y), yp==y))
I = lambda x,y: And(x<=y, y<=n)

F1 = Implies(P, I(x,y))
F2 = Implies(And(I(x,y), B, T), I(xp,yp))
F3 = Implies(And(Not(B), I(x,y)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [48]:
#   loop-zilu/benchmark06_conjunctive.c
#   j=0;
#   if (!(x+y==k)) return 0;
#   while (__VERIFIER_nondet_bool()) {
#     if(j==i) {x++;y--;} else {y++;x--;} j++;
#   }
#   __VERIFIER_assert(x+y==k);

i = Int('i')
j = Int('j')
x = Int('x')
y = Int('y')
k = Int('k')
ip = Int('ip')
jp = Int('jp')
xp = Int('xp')
yp = Int('yp')

P = And(j == 0, x + y == k)
Q = x+y==k
T = And(Implies(j==i, And(xp==x+1, yp==y-1)), Implies(Not(j==i), And(yp==y+1, xp==x-1)), jp==j+1)
I = lambda i,j,x,y: x+y==k

F1 = Implies(P, I(i,j,x,y))
F2 = Implies(And(I(i,j,x,y), T), I(ip,jp,xp,yp))
F3 = Implies(I(i,j,x,y), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [8]:
# loop-zilu/benchmark07_linear.c; 
# modified to make it provable
#   _Bool flag = __VERIFIER_nondet_bool();
#   i=0;
#   if (!(n>0 && n<10)) return 0;
#   if (!(k>0)) return 0; // added by us; otherwise can't prove
#   while (i<n) {
#     i++;
#     if(flag) k+=4000;
#     else k+=2000;
#   }
#   __VERIFIER_assert(k>n);

i = Int('i')
j = Int('j')
n = Int('n')
k = Int('k')
ip = Int('ip')
kp = Int('kp')
flag = Bool('flag')

P = And(n>0, n<10, i==0, k>0)
Q = k>n
B = i<n
T = And(ip==i+1, Implies(flag, kp==k+4000), Implies(Not(flag), kp==k+2000))
I = lambda i,k: k>i

F1 = Implies(P, I(i,k))
F2 = Implies(And(I(i,k), B, T), I(ip,kp))
F3 = Implies(And(Not(B), I(i,k)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [15]:
# loop-zilu/benchmark13_conjunctive.c
#   if (!(i==0 && j==0)) return 0;
#   while (i <= k) {
#     i++;
#     j=j+1;
#   }
#   __VERIFIER_assert(j==i);

i = Int('i')
j = Int('j')
k = Int('k')
ip = Int('ip')
jp = Int('jp')

P = And(i == 0, j == 0)
Q = And(j == i)
B = And(i <= k)
T = And(i == ip + 1, j == jp + 1)
I = lambda i, j: And((-1*i + 1*j + 0*k <= 0),(1*i + -1*j + 0*k <= 0))
# And(i == j)

F1 = Implies(P, I(i, j))
F2 = Implies(And(I(i, j), B, T), I(ip, jp))
F3 = Implies(And(I(i, j), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [46]:
# loop-zilu/benchmark15_conjunctive.c
#   if (!(low == 0 && mid >= 1 && high == 2*mid)) return 0;
#   while (mid > 0) {
#     low = low + 1;
#     high = high - 1;
#     mid = mid - 1;
#   }
#   __VERIFIER_assert(low == high);

low = Int('low')
mid = Int('mid')
high = Int('high')
lowp = Int('lowp')
midp = Int('midp')
highp = Int('highp')

P = And(low == 0, mid >= 1, high == 2*mid)
Q = low == high
B = mid > 0
T = And(lowp == low + 1, highp == high - 1, midp == mid - 1)
I = lambda low, mid, high: And(low+2*mid == high, mid>=0)

F1 = Implies(P, I(low, mid, high))
F2 = Implies(And(I(low, mid, high), B, T), I(lowp, midp, highp))
F3 = Implies(And(I(low, mid, high), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [14]:
#   loop-zilu/benchmark16_conjunctive.c
#   if (!(0 <= k && k <= 1 && i == 1)) return 0;
#   while (__VERIFIER_nondet_bool()) {
#     i = i + 1;
#     k = k - 1;
#   }
#   __VERIFIER_assert(1 <= i + k && i + k <= 2 && i >= 1);

i = Int('i')
k = Int('k')
ip = Int('ip')
kp = Int('kp')

P = And(0 <= k, k <= 1, i == 1)
Q = And(1 <= i + k, i + k <= 2, i >= 1)
T = And(ip == i + 1, kp == k - 1)
I = lambda i, k: And((-2*i + 1*k <= -1) , (-1*i + -1*k <= -1) , (1*i + 1*k <= 2))
#  And(1 <= i + k, i + k <= 2, i >= 1)

F1 = Implies(P, I(i, k))
F2 = Implies(And(I(i,k), T), I(ip, kp))
F3 = Implies(I(i,k), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [13]:
# loop-zilu/benchmark18_conjunctive.c
#   if (!((i==0) && (k==0) && (n>0))) return 0;
#   while (i < n) {
#     i++;k++;
  # }
  # __VERIFIER_assert((i == k) && (k == n));
    
i = Int('i')
k = Int('k')
n = Int('n')
ip = Int('ip')
kp = Int('kp')

P = And(i==0, k==0, n>0)
B = i < n
Q = And(i==k, k==n)
T = And(ip==i+1, kp==k+1)
I = lambda i, k: And((-1*i + 1*k + 0*n <= 0) , (2*i + -1*k + -1*n <= 0) , (1*i + -1*k + 0*n <= 0))
# And(i==k, i<=n)

F1 = Implies(P, I(i, k))
F2 = Implies(And(I(i,k), B, T), I(ip, kp))
F3 = Implies(And(I(i,k), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [19]:
# /loop-zilu/benchmark19_conjunctive.c
#   if (!((j==n) && (k==n) && (n>0))) return 0;
#   while (j>0 && n>0) {
#     j--;k--;
#   }
#   __VERIFIER_assert((k == 0));

j = Int("j")
k = Int("k")
n = Int("n")
jp = Int("jp")
kp = Int("kp")

P = And(j == n, k == n, n > 0)
B = And(j > 0, n > 0)
Q = k == 0
T = lambda j, k, jp, kp: And(jp==j - 1, kp==k - 1)
I = lambda j,k,n: And(j==k, j>=0, n > 0)

F1 = Implies(P, I(j, k, n))
F2 = Implies(And(B, I(j, k, n), T(j, k, jp, kp)), I(jp, kp, n))
F3 = Implies(And(Not(B), I(j, k, n)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [25]:
# loop-zilu/benchmark22_conjunctive.c
#   if (!(x==1 && y==0)) return 0;
#   while (__VERIFIER_nondet_bool()) {
#     x=x+y;
#     y++;
#   }
#   __VERIFIER_assert(x >= y);

x = Int('x')
y = Int('y')
xp = Int('xp')
yp = Int('yp')

P = And(x==1, y==0)
Q = x >= y
T = And(xp == x + y, yp == y + 1)
I = lambda x, y: And(x >= y, x>=1, y >= 0)

F1 = Implies(P, I(x, y))
F2 = Implies(And(I(x, y), T), I(xp, yp))
F3 = Implies(I(x, y), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [ ]:
#    /loop-zilu/benchmark23_conjunctive.c
#   if (!(i==0 && j==0)) return 0;
#   while (i<100) {
#     j+=2;
#     i++;
#   }
#   __VERIFIER_assert(j==200);

i = Int("i")
j = Int("j")
ip = Int("jp")
jp = Int("kp")

P = And(j == 0, i == 0)
B = And(i < 100)
Q = j == 200
T = lambda i, j, ip, jp: And(jp==j + 2, ip==i + 1)

I = lambda i, j: And(j==2*i, i<=100)

solver = Solver()
solver.add(Not(Implies(P, I(i, j))))
print(solver.check())

solver = Solver()
solver.add(Not(Implies(And(B, I(i, j), T(i, j, ip, jp)), I(ip, jp))))
print(solver.check())

solver = Solver()
solver.add(Not(Implies(And(Not(B), I(i, j)), Q)))
print(solver.check())

In [ ]:
# loop-zilu/benchmark24_conjunctive.c
#   if (!(i==0 && k==n && n>=0)) return 0;
#   while (i<n) {
#     k--;
#     i+=2;
#   }
#   __VERIFIER_assert(2*k>=n-1);

i = Int('i')
k = Int('k')
n = Int('n')
ip = Int('ip')
kp = Int('kp')

P = And(i==0, k==n, n>=0)
Q = And(2*k>=n-1)
B = And(i<n)
T = And(ip == i+2, kp == k-1)
I = lambda i, k, n: And( 2*n <= i + 2*k, i<=2+2*k)
# I = lambda i, k, n: And( 2*n <= i + 2*k, i-n<=1)
# I = lambda i, k, n: And( 2*n <= i + 2*k, 2 * (i - k) <= 4 + n)

F1 = Implies(P, I(i, k, n))
F2 = Implies(And(I(i, k, n), B, T), I(ip, kp, n)) 
F3 = Implies(And(I(i, k, n), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [32]:
# loop-zilu/benchmark28_linear.c
# The original program is incorrect. We add a pre condition to make it correct.
#   if (!(i * i < j * j)) return 0;
#   if (!(j >= 0)) return 0; // added by us
#   while (i < j) {
#     j = j - i;
#     if (j < i) {j = j + i;
#     i = j - i;
#     j = j - i;}
#   }
#   __VERIFIER_assert(j == i);

i = Int('i')
j = Int('j')
ip = Int('ip')
j1 = Int('j1')
j2 = Int('j2')
j3 = Int('j3')

P = And(Or(And(i-j<0, i+j>0), And(i-j>0, i+j<0)), j>=0) 
Q = j == i
B = i < j
T = And(j1 == j - i, Implies(j1 < i, And(j2 == j1 + i, ip == j2 - i, j3 == j2 - ip)), Implies(j1 >= i, And(j1 == j, ip == i, j2 == j1, j3 == j2)))
I = lambda i, j: And(i<=j)

F1 = Implies(P, I(i, j))
F2 = Implies(And(I(i, j), B, T), I(ip, j3))
F3 = Implies(And(I(i, j), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [41]:
# loop-zilu/benchmark34_conjunctive.c
#   if (!((j==0) && (k==n) && (n>0))) return 0;
#   while (j<n && n>0) {
#     j++;k--;
#   }
#   __VERIFIER_assert((k == 0));

j = Int('j')
k = Int('k')
n = Int('n')
jp = Int('jp')
kp = Int('kp')

P = And(j == 0, k == n, n > 0)
B = And(j < n, n > 0)
Q = k == 0
T = And(jp == j + 1, kp == k - 1)
I = lambda j, k, n: And(j+k == n, j>=0, k>=0)

F1 = Implies(P, I(j, k, n))
F2 = Implies(And(B, I(j, k, n), T), I(jp, kp, n))
F3 = Implies(And(Not(B), I(j, k, n)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [14]:
# loop-zilu/benchmark41_conjunctive.c
#   if (!(x == y && y == 0 && z==0)) return 0;
#   while (__VERIFIER_nondet_bool()) {
#     x++;y++;z-=2;
#   }
#   __VERIFIER_assert(x == y && x >= 0 && x+y+z==0);

x = Int('x')
y = Int('y')
z = Int('z')
xp = Int('xp')
yp = Int('yp')
zp = Int('zp')

P = And(x == y, y == 0, z==0)
Q = And(x == y, x >= 0, x+y+z==0)
B = True
T = And(xp == x + 1, yp == y + 1, zp == z - 2)
I = lambda x, y, z: And(x - y == 0,x + y + z == 0,-x + 2*z <= 3)
# And(x == y, x + y + z == 0, x >= 0)

F1 = Implies(P, I(x, y, z))
F2 = Implies(And(I(x, y, z), B, T), I(xp, yp, zp))
F3 = Implies(I(x, y, z), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [15]:
# loop-zilu/benchmark44_disjunctive.c
#   if (!(x<y)) return 0;
#   while (x<y) {
#     if ((x<0 && y<0)) {
#       x=x+7; y=y-10;
#     }
#     if ((x<0 && y>=0)) {
#       x=x+7; y=y+3;
#     } else {
#       x=x+10; y=y+3;
#     }
#   }
#   __VERIFIER_assert(x >= y && x <= y + 16);

x = Int('x')
y = Int('y')
xp = Int('xp')
yp = Int('yp')
xpp = Int('xpp')
ypp = Int('ypp')

P = x < y
Q = And(x >= y, x <= y + 16)
B = x < y
T = And(Implies(And(x < 0, y < 0), And(xp == x + 7, yp == y - 10)),
        Implies(Not(And(x < 0, y < 0)), And(xp == x, yp == y)),
        Implies(And(xp < 0, yp >= 0), And(xpp == xp + 7, ypp == yp + 3)),
        Implies(Not(And(xp < 0, yp >= 0)), And(xpp == xp + 10, ypp == yp + 3)))
I = lambda x, y: And(x <= y + 16)

F1 = Implies(P, I(x, y))
F2 = Implies(And(I(x, y), B, T), I(xpp, ypp))
F3 = Implies(And(Not(B, I(x, y))), Q)

check(F1, F2, F3)    

unsat
sat
[xp = 5, x = -2, xpp = 15, yp = -11, ypp = -8, y = -1]
sat
[x = 0, y = -17]


In [8]:
# loop-zilu/benchmark47_linear.c
#   if (!(x<y)) return 0;
#   while (x<y) {
#     if (x < 0) x = x + 7;
#     else x = x + 10;
#     if (y < 0) y = y - 10;
#     else y = y + 3;
#   }
#   __VERIFIER_assert(x >= y && x <= y + 16);

x = Int('x')
y = Int('y')
xp = Int('xp')
yp = Int('yp')

P = x < y
B = x < y
Q = And(x >= y, x <= y + 16)
T = And(Implies(x < 0, xp == x + 7), Implies(Not(x < 0), xp == x + 10),
        Implies(y < 0, yp == y - 10), Implies(Not(y < 0), yp == y + 3))
I = lambda x, y: x <= y + 16

F1 = Implies(P, I(x, y))
F2 = Implies(And(I(x, y), B, T), I(xp, yp))
F3 = Implies(And(Not(B), I(x, y)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [11]:
# loop-zilu/benchmark48_linear.c
#   if (!(i<j && k> 0)) return 0;
#   while (i<j) {
#     k=k+1;i=i+1;
#   }
#   __VERIFIER_assert(k > j - i);

i = Int('i')
j = Int('j')
k = Int('k')
ip = Int('ip')
kp = Int('kp')

P = And(i<j, k>0)
Q = k > j - i
B = i < j
T = And(kp == k + 1, ip == i + 1)
I = lambda i, j, k: i-j-k<=-2
# And(k>=1)

F1 = Implies(P, I(i, j, k))
F2 = Implies(And(I(i, j, k), B, T), I(ip, j, kp))
F3 = Implies(And(Not(B), I(i, j, k)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [73]:
# loop-zilu/benchmark53_polynomial.c
#   if (!(x*y>=0)) return 0;
#   while (__VERIFIER_nondet_bool()) {
#     if(x==0)
#     { if (y>0) x++;
#       else x--;} 
#     if(x>0) y++;
#       else x--;
#   }
#   __VERIFIER_assert(x*y>=0);

x = Int('x')
y = Int('y')
xp = Int('xp')
xpp = Int('xpp')
yp = Int('yp')

P = Or(And(x>=0, y>=0), And(x<=0, y<=0))# x*y>=0 # [1, 0, -1, 0], [0, 1, -1, 0]
Q = x*y>=0
B = True
T = And(Implies(x==0, And(Implies(y>0, xp==x+1), Implies(y<=0, xp==x-1))),
        Implies(x!=0, xp==x),
        And(Implies(xp>0, And(yp==y+1, xpp==xp)),
            Implies(xp<=0, And(yp==y, xpp==xp-1))))
I = lambda x, y: Or(And(x>=0, y>=0), And(x<=0, y<=0))
# x*y>=0
# Wrong: Or(And(y<=0, x<=0), And(x-y<=-28, -x+2*y<=9))

F1 = Implies(P, I(x, y))
F2 = Implies(And(I(x, y), B, T), I(xpp, yp))
F3 = Implies(And(Not(B), I(x, y)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


# Other benchmarks

In [41]:
# loop-lit/afnp2014.c
# int x = 1;
# int y = 0;
# while (y < 1000 && __VERIFIER_nondet_int()) {
#     x = x + y;
#     y = y + 1;
# }
# __VERIFIER_assert(x >= y);

x = Int('x')
y = Int('y')
xp = Int('xp')
yp = Int('yp')

P = And(x == 1, y == 0)
B = And(y < 1000, Bool('nondet'))
Q = x >= y
T = And(xp == x + y, yp == y + 1)
I = lambda x, y: And(x >= y, x >= 1, y >= 0)

F1 = Implies(P, I(x, y))
F2 = Implies(And(I(x, y), B, T), I(xp, yp))
F3 = Implies(And(I(x, y), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [33]:
# loop-lit/gsv2008.c
    # int x = -50;
    # if (!(-1000 < y && y < LARGE_INT)) return 0;
    # while (x < 0) {
	# x = x + y;
	# y++;
    # }
    # __VERIFIER_assert(y > 0);

x = Int('x')
y = Int('y')
xp = Int('xp')
yp = Int('yp')

P = And(x == -50, -10000 < y, y < 1000000)
B = x < 0
Q = y > 0
T = And(xp == x + y, yp == y + 1)
I = lambda x, y: Or(y > 0, x < 0)

F1 = Implies(P, I(x, y))
F2 = Implies(And(I(x, y), B, T), I(xp, yp)) # (y>0 or x<0) and x<0 and x'=x+y and y'=y+1 -> y+1>0 or x+y<0
F3 = Implies(And(I(x, y), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [19]:
# loop-lit/cggmp2005.c
#     i = 1;
#     j = 10;
#     while (j >= i) {
#         i = i + 2;
#         j = -1 + j;
#     }
#     __VERIFIER_assert(j == 6);

i = Int('i')
j = Int('j')
ip = Int('ip')
jp = Int('jp')
	# (i<=j)
	# (i+2*j==21)
P = And(i==1, j == 10)
B = j >= i
Q = j == 6
T = And(ip == i + 2, jp == j - 1)
I = lambda i, j: And(i+2*j==21, j>=6)

F1 = Implies(P, I(i,j))
F2 = Implies(And(I(i,j), B, T), I(ip,jp))
F3 = Implies(And(I(i,j), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [17]:
# loop-lit/hhk2008.c
#     if (!(a <= 1000000)) return 0;
#     if (!(0 <= b && b <= 1000000)) return 0;
#     res = a;
#     cnt = b;
#     while (cnt > 0) {
# 	    cnt = cnt - 1;
# 	    res = res + 1;
#     }
#     __VERIFIER_assert(res == a + b);

a = Int('a')
b = Int('b')
res = Int('res')
cnt = Int('cnt')
resp = Int('resp')
cntp = Int('cntp')

P = And(a <= 1000000, 0 <= b, b <= 1000000, res == a, cnt == b)
Q = res == a+b
B = cnt > 0
T = And(cntp == cnt-1, resp == res+1)
I = lambda cnt, res: And(res +cnt== a+b, cnt>=0)

F1 = Implies(P, I(cnt, res))
F2 = Implies(And(I(cnt, res), B, T), I(cntp, resp))
F3 = Implies(And(I(cnt, res), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [10]:
# loop-lit/jm2006_variant.c
# if (!(i >= 0 && i <= LARGE_INT)) return 0;
# if (!(j >= 0)) return 0;
# int x = i;
# int y = j;
# int z = 0;
# while(x != 0) {
#     x --;
#     y -= 2;
#     z ++;
# }
# if (i == j) {
#     __VERIFIER_assert(y == -z);
# }

i = Int('i')
j = Int('j')
x = Int('x')
y = Int('y')
z = Int('z')
xp = Int('xp')
yp = Int('yp')
zp = Int('zp')

P = And(i >= 0, i <= 100, j >= 0, x == i, y == j, z == 0)
B = x != 0
Q = Implies(i == j, y == -z)
T = And(xp == x - 1, yp == y - 2, zp == z + 1)
I = lambda x, y, z: And(y + z == j - (x - i))


F1 = Implies(P, I(x,y,z))
F2 = Implies(And(I(x,y,z), B, T), I(xp,yp,zp)) 
F3 = Implies(And(I(x,y,z), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat


In [7]:
# loop-lit/jm2006.c
    # if (!(i >= 0 && j >= 0)) return 0;
    # int x = i;
    # int y = j;
    # while(x != 0) {
    #     x--;
    #     y--;
    # }

    # if (i == j) {
    #     __VERIFIER_assert(y == 0);
    # }

x = Int('x')
y = Int('y')
i = Int('i')
j = Int('j')
xp = Int('xp')
yp = Int('yp')

P = And(i >= 0, j >= 0, x == i, y == j)
T = And(xp == x - 1, yp == y - 1)
Q = Implies(i == j, y == 0)
B = x != 0
I = lambda x, y, i, j: And(i - x == j - y)

F1 = Implies(P, I(x,y,i,j))
F2 = Implies(And(I(x,y,i,j), B, T), I(xp,yp,i,j)) 
F3 = Implies(And(I(x,y,i,j), Not(B)), Q)

check(F1, F2, F3)

unsat
unsat
unsat
